In [191]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
import random

# Data

Reading the data from the files.

In [192]:
categories = []
category_lines = {}

for (dirpath, dirnames, filenames) in os.walk('data/names'):
    for (i, filename) in enumerate([dirpath + os.sep + f for f in filenames]):
        category = os.path.splitext(os.path.basename(filename))[0]
        categories.append(category)
        lines = open(filename, encoding='utf-8').read().lower().strip().split('\n')
        category_lines[category] = lines


def str_to_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, 512)
    for (i, c) in enumerate(name):
        tensor[i] = F.one_hot(torch.Tensor([ord(c)]).long(), num_classes=512)

    return tensor

# Neural Network Module

Defining the neural network module

In [193]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, bias=True):
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        
        self.ft = nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias=self.bias)
        self.it = nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias=self.bias)
        self.ct = nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias=self.bias)
        self.ot = nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias=self.bias)
        
        self.h2o = nn.Linear(self.hidden_size, output_size, bias=self.bias)
        
        self.softmax = nn.LogSoftmax(dim=1)
        
        
    def forward(self, x, h, c):
        x_h = torch.cat((x, h), dim=1)
        self.ft(x_h)
        forget_gate = F.sigmoid(self.ft(x_h))
        input_gate = F.sigmoid(self.it(x_h)) * F.tanh(self.ct(x_h))
        cell_state = forget_gate * c + input_gate
        hidden_state = F.sigmoid(self.ot(x_h)) * F.tanh(cell_state)
        output = self.softmax(self.h2o(hidden_state))
        return cell_state, hidden_state, output
        
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
    def init_cell_state(self):
        return torch.zeros(1, self.hidden_size)

In [194]:
n_hidden = 128
lstm = LSTMCell(512, n_hidden, len(categories))

# Training

In [195]:
criterion = nn.NLLLoss()
learning_rate = 0.005
n_iters = 100000

In [196]:
def get_training_example():
    category = random.choice(list(categories))
    line = random.choice(category_lines[category])
    category_tensor = torch.tensor([categories.index(category)], dtype=torch.long)
    line_tensor = str_to_tensor(line)
    return category_tensor, line_tensor

In [197]:
def train(category_tensor, line_tensor):
    hidden = lstm.init_hidden()
    cell_state = lstm.init_cell_state()
    
    lstm.zero_grad()
    for i in range (line_tensor.size()[0]):
        cell_state, hidden, output = lstm(line_tensor[i], hidden, cell_state)
        
    loss = criterion(output, category_tensor)
    loss.backward()
    
    for p in lstm.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return output, loss.item()

In [198]:
current_loss = 0
for iter in range(1, n_iters + 1):
    category_tensor, line_tensor = get_training_example()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

# Evaluation

In [250]:
line_tensor = str_to_tensor("Lombard")

hidden = lstm.init_hidden()
cell_state = lstm.init_cell_state()

for i in range (line_tensor.size()[0]):
    cell_state, hidden, output = lstm(line_tensor[i], hidden, cell_state)
        
print(categories[output.argmax().item()])

Dutch
